In [2]:
import pandas as pd
import numpy as np
import os
import requests
import json


In [3]:
def data_import():
    """
    Import der Daten aus allen Dateien, die mit 'waqi-covid-' anfangen.
    Entfernen von Duplikaten und Umbenennung bestimmter Spaltenwerte.
    Zusammenführung der DataFrames.
    """
    data_folder = './data/'
    all_files = [f for f in os.listdir(data_folder) if f.startswith('waqi-covid-') and f.endswith('.csv') or f == 'airquality-covid19-cities.json']
    dataframes = []

    if not all_files:
        print("Keine Dateien gefunden.")
        return None

    for file in all_files:
        file_path = os.path.join(data_folder, file)
        try:
            df = pd.read_csv(file_path, comment='#')

            if "Specie" not in df.columns:
                print(f"Spalte 'Specie' fehlt in {file}")
                continue

            df["Specie"] = df["Specie"].replace("wind gust", "wind-gust")
            df["Specie"] = df["Specie"].replace("wind speed", "wind-speed")
            df = df.drop_duplicates()

            if df.empty:
                print(f"{file} enthält nach Duplikat-Entfernung keine Daten mehr.")
                continue

            dataframes.append(df)
        except Exception as e:
            print(f"Fehler beim Verarbeiten von {file}: {e}")

    if not dataframes:
        print("Keine gültigen Daten vorhanden.")
        return None

    return pd.concat(dataframes, ignore_index=True)

In [4]:
df=data_import()
print(df)

Spalte 'Specie' fehlt in airquality-covid19-cities.json
waqi-covid-2022Q1.csv enthält nach Duplikat-Entfernung keine Daten mehr.
                Date Country    City Specie  count   min    max  median  \
0         2015-01-06      KR  Jeonju     co    124   0.1   12.3     4.5   
1         2015-01-22      KR  Jeonju     co    116   4.5   10.0     6.7   
2         2015-03-30      KR  Jeonju     co    118   1.2   11.2     5.6   
3         2015-05-27      KR  Jeonju     co     93   2.3    5.6     3.4   
4         2015-02-03      KR  Jeonju     co    133   4.5   13.4     7.8   
...              ...     ...     ...    ...    ...   ...    ...     ...   
14251935  2024-08-12      IR  Tehran    so2    154   6.0  101.0    12.0   
14251936  2025-01-13      IR  Tehran    so2    516  10.0   74.0    19.0   
14251937  2024-02-10      IR  Tehran    so2    299   5.0   76.0    34.0   
14251938  2024-05-16      IR  Tehran    so2    559   6.0   92.0    17.0   
14251939  2025-01-04      IR  Tehran    so2   

In [5]:
def add_geodata(df):
    """
    Fügt die Geodaten zu den Städten hinzu
    """
    df = df.copy()

    # JSON-Datei laden
    with open('./data/airquality-covid19-cities.json', 'r', encoding='utf-8') as file:
        geodata = json.load(file)
    
    geodata = geodata["data"] 

    # Erstellen eines DataFrames mit Städten und Geokoordinaten
    df_places = pd.DataFrame([
        {
            "City": entry["Place"]["name"],  # Stadtname
            "Latitude": entry["Place"]["geo"][0],  
            "Longitude": entry["Place"]["geo"][1]
        }
        for entry in geodata if "Place" in entry])  # Nur Einträge mit "Place" verwenden
    

    # Standardisiere den Stadtnamen für eine bessere Übereinstimmung
    df["City"] = df["City"].str.lower().str.strip()
    df_places["City"] = df_places["City"].str.lower().str.strip()

    # Zusammenführen der beiden DataFrames über "City"
    df = df.merge(df_places, on="City", how="left")

    return df

In [ ]:
# def data_cleaning(df):
#     """Bereinigung der Daten
#     - nicht benötigte Spalten löschen
#     - Eine Stadt pro Land mit den meisten Messwerten 
#     - und die Liste als csv ins Datenverzeichnis speichern
#     - Zusammenfassung der Daten nach Datum, Land, Stadt und Spezies, so dass nur ein Messwert je Species (Median) pro Tag/ Stadt verbleibt
#     - filtern des df nach den ausgewählten Städten
#     - Spalte Species aufteilen
#     - df als csv speichern im Datenverzeichnis
#     """
#     df = df.copy()

#     df = df.drop(columns=['variance', 'min', 'max'], errors='ignore')

#     # city_counts = df.groupby(["Country", "City"]).size().reset_index(name="count")
#     # cities = city_counts.loc[city_counts.groupby("Country")["count"].idxmax()]
  
#     # output_path = './data/city_per_country.csv'
#     # os.makedirs(os.path.dirname(output_path), exist_ok=True)
#     # cities.to_csv(output_path, index=False)
#     # print(f"✅ Datei wurde gespeichert: {output_path}")
    
#     # cities=cities['City'].tolist()
#     # df = df[df['City'].isin(cities)]

#     df = df.groupby(["Date", "Country", "City", "Specie"], as_index=False).agg({"median": "mean"})  

#     df = df.pivot(index=["Date", "Country", "City"], columns="Specie", values='median').reset_index()

#     output_path = './data/cleaned_data.csv'
#     os.makedirs(os.path.dirname(output_path), exist_ok=True)
#     df.to_csv(output_path, index=False)
#     print(f"✅ Datei wurde gespeichert: {output_path}")

#     return df



In [6]:
def data_cleaning(df):
    """Bereinigung der Daten
    - nicht benötigte Spalten löschen
    - Zusammenfassung der Daten nach Datum, Land, Stadt und Spezies, so dass nur ein Messwert je Species (Median) pro Tag/ Stadt verbleibt
    - Spalte Species aufteilen
    - df als csv speichern im Datenverzeichnis
    """
    df = df.copy()

    df = df.drop(columns=['variance', 'min', 'max'], errors='ignore')

    df = df.groupby(["Date", "Country", "City", "Specie"], as_index=False).agg({"median": "mean"})  

    df = df.pivot(index=["Date", "Country", "City"], columns="Specie", values='median').reset_index()


    output_path = './data/cleaned_data.csv'
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    df.to_csv(output_path, index=False)
    print(f"✅ Datei wurde gespeichert: {output_path}")


    df= add_geodata(df)


    return df



In [7]:
df=data_cleaning(df)
print(df.shape)
df.head()  

✅ Datei wurde gespeichert: ./data/cleaned_data.csv
(1555645, 27)


,Date,Country,City,aqi,co,d,dew,humidity,mepaqi,neph,...,pressure,psi,so2,temperature,uvi,wd,wind-gust,wind-speed,Latitude,Longitude
0,2014-12-29,AT,graz,NaN,0.1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.6,NaN,NaN,NaN,NaN,NaN,47.06667,15.45000
1,2014-12-29,AT,innsbruck,NaN,0.1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.6,NaN,NaN,NaN,NaN,NaN,47.26266,11.39454
2,2014-12-29,AT,linz,NaN,0.1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2.1,NaN,NaN,NaN,NaN,NaN,48.30639,14.28611
3,2014-12-29,AT,salzburg,NaN,0.1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2.1,NaN,NaN,NaN,NaN,NaN,47.79941,13.04399
4,2014-12-29,AT,vienna,NaN,0.1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2.6,NaN,NaN,NaN,NaN,NaN,48.20849,16.37208


In [ ]:
cities = df[['City', 'Latitude', 'Longitude']]
print(cities.head())

In [ ]:
from meteostat import Stations

# Test mit einer Stadt
city = {'City': 'Berlin', 'Latitude': 52.5200, 'Longitude': 13.4050}

# Nächste Wetterstation suchen
stations = Stations().nearby(city['Latitude'], city['Longitude'])
station = stations.fetch(5)  # Hol dir mal die nächsten 5 Stationen zum Vergleich

print(station)


In [8]:
from meteostat import Daily
from datetime import datetime

# Test mit der ersten Station
start = datetime(2023, 1, 1)
end = datetime(2023, 1, 31)

if not station.empty:
    station_id = station.index[0]
    print(f"📡 Verwende Station: {station_id}")

    # Tägliche Wetterdaten abrufen
    data = Daily(station_id, start, end)
    data = data.fetch()

    print(data.head())
else:
    print("⚠️ Keine Wetterstation gefunden!")

NameError: name 'station' is not defined

In [ ]:
#WEtterdaten integrieren



from meteostat import Daily, Stations
from datetime import datetime

# Städte
cities = df[['City', 'Latitude', 'Longitude']].drop_duplicates()

# Zeitspanne festlegen
start = datetime(2015, 1, 1)
end = datetime(2024, 12, 31)

# DataFrame für alle Städte vorbereiten
all_data = pd.DataFrame()

# Daten für jede Stadt abrufen und hinzufügen
for _, city in cities.iterrows():
    # Nächste Wetterstation suchen
    stations = Stations().nearby(city['Latitude'], city['Longitude'])
    station = stations.fetch(1)

    if not station.empty:
        station_id = station.index[0]

        # Tägliche Wetterdaten abrufen
        data = Daily(station_id, start, end).fetch()

        # Nan-Daten rausfiltern
        data.dropna(how='all', inplace=True)

        if not data.empty:
            # Stadtname hinzufügen
            data["City"] = city["City"]

            # Daten in den Gesamtdaten-Frame einfügen
            all_data = pd.concat([all_data, data])

    # if station.empty:
    #     print(f"⚠️ Keine Wetterstation für {city['City']} gefunden, übersprungen.")
    #     continue

    # # Tägliche Wetterdaten abrufen
    # data = Daily(station.index[0], start, end)
    # data = data.fetch()

    # if data.empty:
    #     print(f"⚠️ Keine Wetterdaten für {city['City']} gefunden, übersprungen.")
    #     continue

    # # Stadtname hinzufügen
    # data["City"] = city["City"]

    # # Daten in den Gesamtdaten-Frame einfügen
    # all_data = pd.concat([all_data, data])

# Index zurücksetzen
all_data.reset_index(inplace=True)

# Spalte time umbennen in Date
all_data.rename(columns={'time': 'Date'}, inplace=True) 

# Standardisiere den Stadtnamen für eine bessere Übereinstimmung
all_data["City"] = all_data["City"].str.lower().str.strip()

print(f"✅ Wetterdaten gesammelt für {all_data['City'].nunique()} Städte")

# # Ergebnis anzeigen
# print(all_data.head())

✅ Wetterdaten gesammelt für 577 Städte


In [ ]:
# #print(df[['City', 'Date']].head())
# print(all_data.head())

In [ ]:
# print(df['Date'].dtype)
# print(all_data['Date'].dtype)
# print(df['City'].dtype)
# print(all_data['City'].dtype)

In [ ]:
# print(df['City'].unique())
# print(all_data['City'].unique())

# print(set(df['Date']).difference(set(all_data['Date'])))
# print(set(df['City']).difference(set(all_data['City'])))

In [10]:

# print(type(df))
# print(type(all_data))

# print(df.shape)
# print(all_data.shape)


# Konvertiere die 'Date' Spalte in beiden DataFrames zu datetime
df['Date'] = pd.to_datetime(df['Date']).dt.strftime('%Y-%m-%d')
all_data['Date'] = pd.to_datetime(all_data['Date']).dt.strftime('%Y-%m-%d')

#print(len(set(df['Date']).difference(set(all_data['Date']))))

In [ ]:
print(all_data.sample(15))


In [ ]:
# # Funktion für Übersicht über dtypes, missing values, unique values etc.
# def overview(df):
#     '''
#     Erstelle einen Überblick über einige Eigenschaften der Spalten eines DataFrames.
#     VARs
#         df: Der zu betrachtende DataFrame
#     RETURNS:
#         None
#     '''
#     display(pd.DataFrame({'dtype': df.dtypes,
#                           'total': df.count(),
#                           'missing': df.isna().sum(),
#                           'missing%': df.isna().mean()*100,
#                           'n_uniques': df.nunique(),
#                           'uniques%': df.nunique()/df.shape[0]*100,
#                           'uniques': [df[col].unique() for col in df.columns]
#                          }))
# overview(all_data)

In [ ]:
#all_data.columns

In [11]:
# Berechne den Anteil der NaN-Werte pro Spalte
missing_percentage = all_data.isna().mean() * 100

# Lösche die Spalten, bei denen der Anteil an NaN-Werten größer als 80% ist
all_data = all_data.loc[:, missing_percentage <= 80]

In [ ]:
#all_data.columns

In [12]:


# Zusammenführen der beiden DataFrames über "City"
df = pd.merge(df, all_data, on=["City", 'Date'],how="left")
print(df.shape)
df.sample(30)

(1565693, 34)


,Date,Country,City,aqi,co,d,dew,humidity,mepaqi,neph,...,wind-speed,Latitude,Longitude,tavg,tmin,tmax,prcp,wdir,wspd,pres
115613,2016-03-08,EC,quito,NaN,8.9,NaN,NaN,NaN,NaN,NaN,...,NaN,-0.22985,-78.52495,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1452835,2024-08-17,CA,winnipeg,NaN,NaN,NaN,15.0,78.0,NaN,NaN,...,3.0,49.88440,-97.14704,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1195153,2023-02-13,NZ,hamilton,NaN,NaN,NaN,NaN,97.0,NaN,NaN,...,4.0,43.25011,-79.84963,2.6,-1.4,9.1,0.0,252.0,20.2,1005.5
8030,2015-01-17,SV,san salvador,NaN,NaN,NaN,NaN,NaN,33.2,NaN,...,NaN,13.68935,-89.18718,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1095905,2022-08-23,GB,belfast,NaN,0.6,NaN,14.0,79.5,NaN,NaN,...,2.3,54.59682,-5.92541,18.6,16.0,22.0,2.6,203.0,9.1,1009.1
1476365,2024-09-27,KR,busan,NaN,3.2,NaN,20.0,78.0,NaN,NaN,...,2.0,35.10168,129.03004,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82895,2015-07-04,FR,besançon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,47.24878,6.01815,29.3,21.5,35.9,NaN,NaN,6.7,1019.2
1286793,2023-08-12,VN,haiphong,NaN,7.0,NaN,25.5,85.7,NaN,NaN,...,2.0,20.86481,106.68345,27.8,23.6,32.3,15.0,59.0,6.9,1001.8
122472,2016-03-23,AU,adelaide,NaN,0.1,NaN,NaN,NaN,NaN,NaN,...,NaN,-34.92866,138.59863,NaN,17.7,25.3,0.0,NaN,NaN,NaN
1239195,2023-04-30,KR,jeju city,NaN,2.3,NaN,6.0,48.0,NaN,NaN,...,3.1,33.50972,126.52194,17.1,10.0,18.9,0.0,258.0,22.2,1013.8


In [13]:
df.columns

Index(['Date', 'Country', 'City', 'aqi', 'co', 'd', 'dew', 'humidity',
       'mepaqi', 'neph', 'no2', 'o3', 'pm1', 'pm10', 'pm25', 'pol',
       'precipitation', 'pressure', 'psi', 'so2', 'temperature', 'uvi', 'wd',
       'wind-gust', 'wind-speed', 'Latitude', 'Longitude', 'tavg', 'tmin',
       'tmax', 'prcp', 'wdir', 'wspd', 'pres'],
      dtype='object')

In [15]:
print(df.sample(5).T)
#df.isna().sum()

                  6761        813443      443505      1212011     372838 
Date           2015-01-14  2021-02-13  2019-06-11  2023-03-15  2019-02-05
Country                MY          CA          NL          AT          ZA
City              malacca      québec       breda   innsbruck  middelburg
aqi                  38.0         NaN         NaN         NaN         NaN
co                    NaN         0.2         NaN         0.1         2.9
d                     NaN         NaN         NaN         NaN         NaN
dew                   NaN       -20.0        10.0        -3.8        13.9
humidity              NaN        72.1        69.2        60.7         0.0
mepaqi                NaN         NaN         NaN         NaN         NaN
neph                  NaN         NaN         NaN         NaN         NaN
no2                   NaN         6.0         7.3         5.5         4.2
o3                    NaN        20.3        21.1         NaN        13.9
pm1                   NaN         NaN 